In [27]:
import serial
import tkinter
from tkinter import *
import folium as fl
import numpy as np
import io
import selenium
from PIL import Image
import cv2
from folium.plugins import MousePosition
from PIL import Image, ImageTk
import time

Seting up the map (Takes about 80 seconds)

In [28]:
arduino = serial.Serial(port='COM5', baudrate=9600, timeout=.1) 

In [29]:
#Sends message to bluetooth over Serial comm, saves response
def write_read(x): 
	arduino.write(bytes(x, 'utf-8')) 
	time.sleep(0.05) 
	data = arduino.readline() 
	return data 

#sends whatever to the ardy
def write(x):
    arduino.write(bytes(x, 'utf-8')) 
    time.sleep(0.05)

#Gets GPS Coordinates from Arduino
def getLoc():
    #Gets the current GPS location from the ardy
    loc=write_read("Refreshing position")
    loc=str(loc,'utf-8')
    loc=write_read("Asking for coordinates")
    loc=str(loc,'utf-8')
    print("Robot GPS Coordinates:",loc)
    lat=float(loc[:loc.find(",")])
    lng=float(loc[loc.find(",")+1:])
    coords=[lat,lng]
    return coords

In [30]:
startingLoc=getLoc()
targetLoc=(startingLoc[0],startingLoc[1])
view=2
vertRange=view*4
horzRange=view*8
m2c=1/69
boundSW=(targetLoc[0]-np.sqrt(8)/(69*5280),targetLoc[1]-np.sqrt(8)/(69*5280))
boundNE=(targetLoc[0]+np.sqrt(8)/(69*5280),targetLoc[1]+np.sqrt(8)/(69*5280))

Robot GPS Coordinates: 38.816368,-76.880790



In [31]:
m = fl.Map(location=targetLoc)
m.fit_bounds([boundSW,boundNE])
MousePosition().add_to(m)

In [32]:
img_data = m._to_png(1)
img = Image.open(io.BytesIO(img_data))
img.save('image.png')
im=cv2.imread('image.png')
print(im.shape)
centerPix = (im.shape[0]/2,im.shape[1]/2)
centerCoord=targetLoc
print(centerPix)
vp2c=vertRange/im.shape[0]/69
hp2c=horzRange/im.shape[1]/69
print(vp2c,hp2c)

(768, 1366, 3)
(384.0, 683.0)
0.00015096618357487922 0.00016975406879283638


In [33]:
xCoord=0
yCoord=0


In [34]:
image1 = Image.open("image.png")


root = Tk()
f=root.frame()
root.title("Map")
top = Toplevel()
top.title("Robot Controls")
# Create a photoimage object of the image in the path
test = ImageTk.PhotoImage(image1)
label1 = tkinter.Label(image=test)
canvas= Canvas(root, width=test.width(), height=test.height())

canvas.create_image(50, 10, image=test, anchor=NW)
canvas.pack()

T=Text(top)


Gets the computer communicating with the robot

In [36]:
#Sends message to bluetooth over Serial comm, saves response
def write_read(x): 
	arduino.write(bytes(x, 'utf-8')) 
	time.sleep(0.05) 
	data = arduino.readline() 
	return data 

#sends whatever to the ardy
def write(x):
    arduino.write(bytes(x, 'utf-8')) 
    time.sleep(0.05)

#Gets GPS Coordinates from Arduino
def getLoc():
    '''Gets the current GPS location from the ardy'''
    loc=write_read("Refreshing position")
    loc=str(loc,'utf-8')
    loc=write_read("Asking for coordinates")
    loc=str(loc,'utf-8')
    print("Robot GPS Coordinates:",loc)
    lat=float(loc[:loc.find(",")])
    lng=float(loc[loc.find(",")+1:])
    coords=[lat,lng]
    return coords

Sets up the user interface

In [37]:
def key(event):
    print("pressed", repr(event.char))
    
def callback(event):
    canvas.delete("all")
    canvas.create_image(50, 10, image=test, anchor=NW)
    print("clicked at", event.x, event.y)
    xdif=event.x-centerPix[1]
    ydif=event.y-centerPix[1]
    xCoord=centerCoord[0]-(xdif*hp2c)
    yCoord=centerCoord[1]-(ydif*vp2c)
    print("Click GPS is",xCoord,yCoord)
    #canvas.create_oval(event.x, event.y, event.x, event.y, width = 5, fill = 'white')
    canvas.create_text(event.x,event.y,text="GPS Location is "+str(xCoord)+","+str(yCoord))
    T.delete("1.0","end")
    T.insert(END,"Selected position: "+str(xCoord)+','+str(yCoord))
    

#canvas= Canvas(root, width=test.width(), height=test.height())
canvas.bind("<Key>", key)
canvas.place(relx=0,rely=0,anchor=NW)
canvas.bind("<Button-1>", callback)
canvas.create_image(50, 10, image=test, anchor=NW)
canvas.pack()


In [38]:
# Top level window 
# Function for getting Input 
# from textbox and printing it 
# at label widget 

def sendCoords(): 
    print("Test 1")
    loc=getLoc()
    print(loc)
    print("Test 2")
    write_read(str(xCoord)+','+str(yCoord))
    lbl.config(text = "Coordinates sent to robot")
 

# TextBox Creation 
inputtxt = Text(top, 
				height = 5, 
				width = 20) 

inputtxt.pack() 

# Button Creation 
printButton = Button(top, 
						text = "New Coordinates", 
						command = sendCoords) 
printButton.pack() 

'''refreshButton = Button(top, 
						text = "Refresh Map", 
						command = refreshMap) 
refreshButton.pack()'''


# Label Creation 
lbl = Label(top, text = "") 
lbl.pack() 
#T.pack()

In [39]:
root.minsize(test.width(),test.height())
root.mainloop()
arduino.close()

clicked at 778 335
Click GPS is 38.80024136346468 -76.82825376811594
Test 1
Robot GPS Coordinates: 38.816368,-76.880790

[38.816368, -76.88079]
Test 2
clicked at 589 219
Click GPS is 38.83232488246652 -76.81074169082126
clicked at 535 347
Click GPS is 38.841491602181335 -76.83006536231885
